# Group 14
## Competition Name : Moscow Housing
## Elias Elfarri    , ID: 473700
## Nora Valen       , ID: 490606
## Muhammad Sarmad  , ID: 190729

Please note that the estimated run time of our short notebook is 3 hours. 

Predictions can be found under: FirstShortNotebook_submission.csv

PUBLIC LEADERBOARD SCORE: 0.15111

In [ ]:
import json
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 

np.random.seed(123)
sns.set_style('darkgrid')
pd.set_option('display.max_colwidth', None)

!ln -s /kaggle/input/moscow-housing-tdt4173 ./data
!ls ./data | sort

In [ ]:
def root_mean_squared_log_error(y_true, y_pred):
    # Alternatively: sklearn.metrics.mean_squared_log_error(y_true, y_pred) ** 0.5
    assert (y_true >= 0).all() 
    assert (y_pred >= 0).all()
    log_error = np.log1p(y_pred) - np.log1p(y_true)  # Note: log1p(x) = log(1 + x)
    return np.mean(log_error ** 2) ** 0.5

In [ ]:
#Xgboost 1 PIPELINE  
apartments = pd.read_csv('data/apartments_train.csv')
buildings = pd.read_csv('data/buildings_train.csv')
data = pd.merge(apartments, buildings.set_index('id'), how='left', left_on='building_id', right_index=True)

apartments_test = pd.read_csv('data/apartments_test.csv')
buildings_test = pd.read_csv('data/buildings_test.csv')
data_test = pd.merge(apartments_test, buildings_test.set_index('id'), how='left', left_on='building_id', right_index=True)


#Feature Cleaning
maxc = 9
minc = 1
data['ceiling'] = data.apply(lambda row: data["ceiling"].mode()[0] if (row["ceiling"] < minc or row["ceiling"] > maxc ) else( row["ceiling"]) ,axis=1)     
data_test['ceiling'] = data_test.apply(lambda row: data["ceiling"].mode()[0] if (row["ceiling"] < minc or row["ceiling"] > maxc ) else( row["ceiling"]) ,axis=1)     


var =  4.0
data['condition'] = data['condition'].fillna(var)
data_test['condition'] = data_test['condition'].fillna(var)


data['constructed'] = data.apply(
    lambda row: 2019 if (np.isnan(row['constructed']) and ~np.isnan(row['new']) and row['new'] == 0.0) else( 2021 if (np.isnan(row['constructed'])) else row['constructed']),
    axis=1
)  
data['new'] = data.apply(
    lambda row: 0.0 if (np.isnan(row['new']) and row['constructed'] < 2020) else( 1.0 if (np.isnan(row['new'])) else row['new']),
    axis=1
)      

data_test['constructed'] = data_test.apply(
    lambda row: 2019 if (np.isnan(row['constructed']) and ~np.isnan(row['new']) and row['new'] == 0.0) else( 2021 if (np.isnan(row['constructed'])) else row['constructed']),
    axis=1
)  
data_test['new'] = data_test.apply(
    lambda row: 0.0 if (np.isnan(row['new']) and row['constructed'] < 2020) else( 1.0 if (np.isnan(row['new'])) else row['new']),
    axis=1
)     

        
for feature in ["rooms", "area_total", "area_kitchen", "area_living", "floor","latitude", 
                "longitude","district", "seller", "windows_court", "balconies", "material", "stories"]:
    mean = data[feature].mean()
    data[feature] = data[feature].fillna(mean)    
    data_test[feature] = data_test[feature].fillna(mean)


        
        
#FEATURE ENGINEERING DONE:
data['area_total'] = np.log1p(data['area_total'])
data['area_kitchen'] = np.log1p(data['area_kitchen'])
data['area_living'] = np.log1p(data['area_living'])

data_test['area_total'] = np.log1p(data_test['area_total'])
data_test['area_kitchen'] = np.log1p(data_test['area_kitchen'])
data_test['area_living'] = np.log1p(data_test['area_living'])


data['elevatern'] = data.apply(lambda row: 0 if (row["elevator_without"] == 1 and row["elevator_passenger"] == 0.0 and row["elevator_service"] == 0.0 ) # 
                               else( 1 if(row["elevator_without"] == 1 and row["elevator_passenger"] == 0.0 and row["elevator_service"] == 1.0) # 
                                else(2 if(row["elevator_without"] == 1 and row["elevator_passenger"] == 1.0 and row["elevator_service"] == 0.0) # 
                                else(3 if(row["elevator_without"] == 0 and row["elevator_passenger"] == 0.0 and row["elevator_service"] == 1.0) # 
                                else(4 if(row["elevator_without"] == 0 and row["elevator_passenger"] == 1.0 and row["elevator_service"] == 0.0) # 
                                else(5 if(row["elevator_without"] == 0 and row["elevator_passenger"] == 1.0 and row["elevator_service"] == 1.0) # 
                                else(6 if(row["elevator_without"] == 1 and row["elevator_passenger"] == 1.0 and row["elevator_service"] == 1.0)
                                else(np.nan)
                                )   
                                )
                                )
                                )   
                                )
                                )
                                ,axis=1) 
data_test['elevatern'] = data_test.apply(lambda row: 0 if (row["elevator_without"] == 1 and row["elevator_passenger"] == 0.0 and row["elevator_service"] == 0.0 ) # 
                               else( 1 if(row["elevator_without"] == 1 and row["elevator_passenger"] == 0.0 and row["elevator_service"] == 1.0) # 
                                else(2 if(row["elevator_without"] == 1 and row["elevator_passenger"] == 1.0 and row["elevator_service"] == 0.0) # 
                                else(3 if(row["elevator_without"] == 0 and row["elevator_passenger"] == 0.0 and row["elevator_service"] == 1.0) # 
                                else(4 if(row["elevator_without"] == 0 and row["elevator_passenger"] == 1.0 and row["elevator_service"] == 0.0) # 
                                else(5 if(row["elevator_without"] == 0 and row["elevator_passenger"] == 1.0 and row["elevator_service"] == 1.0) # 
                                else(6 if(row["elevator_without"] == 1 and row["elevator_passenger"] == 1.0 and row["elevator_service"] == 1.0)
                                else(np.nan)
                                )      
                                )
                                )
                                )   
                                )
                                )
                                ,axis=1)  
mod = data['elevatern'].mode()
data['elevatern'] = data['elevatern'].fillna(mod[0])
data_test['elevatern'] = data_test['elevatern'].fillna(mod[0])


#Adding city center as origin
origin_coordinates = (37.6, 55.75)
distance_from_city_center = np.sqrt((origin_coordinates[0] - data["longitude"])**2+(origin_coordinates[1] - data["latitude"])**2)
data["distance_from_city_center"] = distance_from_city_center

distance_from_city_center_t = np.sqrt((origin_coordinates[0] - data_test["longitude"])**2+(origin_coordinates[1] - data_test["latitude"])**2)
data_test["distance_from_city_center"] = distance_from_city_center_t




#FEATURES INCLUDED:
features = ["ceiling", "rooms", "area_total", "area_kitchen", "area_living", "floor", "condition","new", "elevatern","distance_from_city_center",
            "latitude", "longitude","district", "constructed", "seller", "windows_court", "balconies", "material", "stories"]



#Model
from xgboost import XGBRegressor

train_x = data[features]
train_y = np.log1p(data['price'])
test_x = data_test[features]

    
model_xgb1 = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.5144984086781564, gamma=0, learning_rate=0.01693820796093592, max_delta_step=0,
       max_depth=23, min_child_weight=5, n_estimators=3977,
       n_jobs=1, nthread=None, objective='reg:squarederror', random_state=2020, # squarederror  reg:squaredlogerror   reg:squarederror
       reg_alpha=0.021096319890667407, reg_lambda=0.2287729489989326, scale_pos_weight=1, seed=None, subsample=0.42023355655422495)


model_xgb1.fit(train_x,train_y)

xgb_preds = model_xgb1.predict(test_x)


In [ ]:
#Catboost PIPELINE  
apartments = pd.read_csv('./data/apartments_train.csv')
buildings = pd.read_csv('./data/buildings_train.csv')
data = pd.merge(apartments, buildings.set_index('id'), how='left', left_on='building_id', right_index=True)

apartments_test = pd.read_csv('./data/apartments_test.csv')
buildings_test = pd.read_csv('./data/buildings_test.csv')
data_test = pd.merge(apartments_test, buildings_test.set_index('id'), how='left', left_on='building_id', right_index=True)


#FEATURE CLEANING:
for feature in ["ceiling","rooms", "area_total", "area_kitchen", "area_living", "floor", "condition","new",
            "latitude", "longitude","district", "constructed", "seller", "balconies", "material", "stories"]:
        if data[feature].max() == 1.0 or feature == "elevator_service" or feature == "condition" or feature == "constructed" or feature == "material" or feature == "seller" :
            #print('Categorical',feature)
            mod = data[feature].mode()
            data[feature] = data[feature].fillna(mod[0])
            
            mod_t = data_test[feature].mode()
            data_test[feature] = data_test[feature].fillna(mod_t[0])          
        else:
            mean = data[feature].mean()
            data[feature] = data[feature].fillna(mean)
            
            mean_t = data_test[feature].mean()
            data_test[feature] = data_test[feature].fillna(mean_t)



#FEATURE ENGINEERING:
origin_coordinates = (37.6, 55.75)
distance_from_city_center = np.sqrt((origin_coordinates[0] - data["longitude"])**2+(origin_coordinates[1] - data["latitude"])**2)
data["distance_from_city_center"] = distance_from_city_center

distance_from_city_center_t = np.sqrt((origin_coordinates[0] - data_test["longitude"])**2+(origin_coordinates[1] - data_test["latitude"])**2)
data_test["distance_from_city_center"] = distance_from_city_center_t

#FEATURES INCLUDED:
features = ["ceiling","rooms", "area_total", "area_kitchen", "area_living", "floor", "condition","new",
            "latitude", "longitude","district", "constructed", "seller", "balconies", "material", "stories", "distance_from_city_center"]


#Model
from catboost import CatBoostRegressor

train_x = data[features]
train_y = np.log1p(data['price'])
test_x = data_test[features]

 
 

param = {
"objective": "RMSE",
'random_state': 2020, 
'learning_rate': 0.027775682386650822, 
'n_estimators': 9561, 
'reg_lambda': 0.02942773134248745, 
'subsample': 0.6452052083779029,
'depth': 8,
'bagging_temperature': 56.77037557663241}


model_cat2 = CatBoostRegressor(**param)  

model_cat2.fit(train_x,train_y,early_stopping_rounds=100,verbose=False)

catboost2_preds = model_cat2.predict(test_x)

In [ ]:
#LGBM 2 PIPELINE
apartments = pd.read_csv('data/apartments_train.csv')
buildings = pd.read_csv('data/buildings_train.csv')
data = pd.merge(apartments, buildings.set_index('id'), how='left', left_on='building_id', right_index=True)

apartments_test = pd.read_csv('data/apartments_test.csv')
buildings_test = pd.read_csv('data/buildings_test.csv')
data_test = pd.merge(apartments_test, buildings_test.set_index('id'), how='left', left_on='building_id', right_index=True)


from lightgbm import LGBMRegressor
import pyproj
from numpy.random import choice


#FEATURE CLEANING
data_test.latitude.iloc[90] = 55.568139
data_test.longitude.iloc[90]= 37.481831
data_test.latitude.iloc[23] = 55.568139
data_test.longitude.iloc[23]= 37.481831
data_test.latitude.iloc[2511] = 55.544066
data_test.longitude.iloc[2511]= 37.482317
data_test.latitude.iloc[6959] = 55.544066
data_test.longitude.iloc[6959]= 37.482317
data_test.latitude.iloc[5090] = 55.544066
data_test.longitude.iloc[5090]= 37.482317
data_test.latitude.iloc[8596] = 55.544066
data_test.longitude.iloc[8596]= 37.482317
data_test.latitude.iloc[2529] = 55.764335
data_test.longitude.iloc[2529]= 37.907556
data_test.latitude.iloc[4719] = 55.765430
data_test.longitude.iloc[4719]= 37.928284
data_test.latitude.iloc[9547] = 55.765430
data_test.longitude.iloc[9547]= 37.928284

data_test.district[data_test.building_id == 3803] = 11
data_test.district[data_test.building_id == 4636] = 11
data_test.district[data_test.building_id == 4412] = 11
data_test.district[data_test.building_id == 926] = 3
data_test.district[data_test.building_id == 4202] = 3
data_test.district[data_test.building_id == 8811] = 3
data_test.district[data_test.building_id == 6879] = 3
data_test.district[data_test.building_id == 5667] = 3
data_test.district[data_test.building_id == 2265] = 5
data_test.district[data_test.building_id == 6403] = 5
data_test.district[data_test.building_id == 7317] = 5
data_test.district[data_test.building_id == 1647] = 5
data_test.district[data_test.building_id == 183] = 5

data.district[data.building_id == 2029] = 0
data.district[data.building_id == 1255] = 0
data.district[data.building_id == 4162] = 5


#cleaning/engineering all elevators as one feature
data['elevatern'] = data.apply(lambda row: 0 if (row["elevator_without"] == 1 and row["elevator_passenger"] == 0.0 and row["elevator_service"] == 0.0 ) # 
                               else( 1 if(row["elevator_without"] == 1 and row["elevator_passenger"] == 0.0 and row["elevator_service"] == 1.0) # 
                                else(2 if(row["elevator_without"] == 1 and row["elevator_passenger"] == 1.0 and row["elevator_service"] == 0.0) # 
                                else(3 if(row["elevator_without"] == 0 and row["elevator_passenger"] == 0.0 and row["elevator_service"] == 1.0) # 
                                else(4 if(row["elevator_without"] == 0 and row["elevator_passenger"] == 1.0 and row["elevator_service"] == 0.0) # 
                                else(5 if(row["elevator_without"] == 0 and row["elevator_passenger"] == 1.0 and row["elevator_service"] == 1.0) # 
                                else(6 if(row["elevator_without"] == 1 and row["elevator_passenger"] == 1.0 and row["elevator_service"] == 1.0)
                                else(np.nan)
                                )    
                                )
                                )
                                )   
                                )
                                )
                                ,axis=1)
data_test['elevatern'] = data_test.apply(lambda row: 0 if (row["elevator_without"] == 1 and row["elevator_passenger"] == 0.0 and row["elevator_service"] == 0.0 ) # 
                               else( 1 if(row["elevator_without"] == 1 and row["elevator_passenger"] == 0.0 and row["elevator_service"] == 1.0) # 
                                else(2 if(row["elevator_without"] == 1 and row["elevator_passenger"] == 1.0 and row["elevator_service"] == 0.0) # 
                                else(3 if(row["elevator_without"] == 0 and row["elevator_passenger"] == 0.0 and row["elevator_service"] == 1.0) # 
                                else(4 if(row["elevator_without"] == 0 and row["elevator_passenger"] == 1.0 and row["elevator_service"] == 0.0) # 
                                else(5 if(row["elevator_without"] == 0 and row["elevator_passenger"] == 1.0 and row["elevator_service"] == 1.0) # 
                                else(6 if(row["elevator_without"] == 1 and row["elevator_passenger"] == 1.0 and row["elevator_service"] == 1.0)
                                else(np.nan)
                                ) 
                                )
                                )
                                )   
                                )
                                )
                                ,axis=1)  
mod = data['elevatern'].mode()
data['elevatern'] = data['elevatern'].fillna(mod[0])
data_test['elevatern'] = data_test['elevatern'].fillna(mod[0])


data["bathrooms_shared"] = data["bathrooms_shared"].fillna(1)
data["bathrooms_private"] = data["bathrooms_private"].fillna(1)
data_test["bathrooms_shared"] = data_test["bathrooms_shared"].fillna(1)
data_test["bathrooms_private"] = data_test["bathrooms_private"].fillna(1)



data["parking"] =  data["parking"].fillna(3.0)
data_test["parking"] =  data_test["parking"].fillna(3.0)
data["heating"] =  data["heating"].fillna(0.0)
data_test["heating"] =  data_test["heating"].fillna(0.0)

#engineering and cleaning a feature
data["total_balconies"] = data["balconies"] + data["loggias"]
data_test["total_balconies"] = data_test["balconies"] + data_test["loggias"]
data["total_balconies"] =  data["total_balconies"].fillna(1.0)
data_test["total_balconies"] =  data_test["total_balconies"].fillna(1.0)


#seller
list_of_candidates = [0,1,2,3]
probability_distribution  = [0.11, 0.33, 0.13, 0.43]
number_of_items_to_pick = data['seller'].isna().sum()
number_of_items_to_pick_test = data_test['seller'].isna().sum()

np.random.seed(0)

draw = choice(list_of_candidates, number_of_items_to_pick,
              p=probability_distribution)
draw_test = choice(list_of_candidates, number_of_items_to_pick_test,
              p=probability_distribution)

data['seller'][data.seller.isna()] = draw
data_test['seller'][data_test.seller.isna()] = draw_test


#area_kitchen and living
percentage_area_data = pd.DataFrame()
percentage_area_data["area_kitchen"] = data["area_kitchen"][data.area_living + data.area_kitchen < data.area_total]/data["area_total"][data.area_living + data.area_kitchen < data.area_total]
percentage_area_data["area_living"] = data["area_living"][data.area_living + data.area_kitchen < data.area_total]/data["area_total"][data.area_living + data.area_kitchen < data.area_total]

mean_kitchen = percentage_area_data["area_kitchen"].mean()
mean_living = percentage_area_data["area_living"].mean()

#to omit bugs
data["area_kitchen_edit"] = data["area_kitchen"].copy()
data["area_living_edit"] = data["area_living"].copy()

data["area_kitchen_edit"][(data.area_living + data.area_kitchen >= data.area_total) | (data.area_living.isna() | data.area_kitchen.isna())] = data.area_total*mean_kitchen
data["area_living_edit"][(data.area_living + data.area_kitchen >= data.area_total) | (data.area_living.isna() | data.area_kitchen.isna())] = data.area_total*mean_living

data["area_kitchen"] = data["area_kitchen_edit"].copy()
data["area_living"] = data["area_living_edit"].copy()

#test_set
data_test["area_kitchen_edit"] = data_test["area_kitchen"].copy()
data_test["area_living_edit"] = data_test["area_living"].copy()

data_test["area_kitchen_edit"][(data_test.area_living + data_test.area_kitchen >= data_test.area_total) | (data_test.area_living.isna() | data_test.area_kitchen.isna())] = data_test.area_total*mean_kitchen
data_test["area_living_edit"][(data_test.area_living + data_test.area_kitchen >= data_test.area_total) | (data_test.area_living.isna() | data_test.area_kitchen.isna())] = data_test.area_total*mean_living


data_test["area_kitchen"] = data_test["area_kitchen_edit"].copy()
data_test["area_living"] = data_test["area_living_edit"].copy()
        

#ceiling    
maxc = 9
minc = 1
data['ceiling'] = data.apply(lambda row: data["ceiling"].mode()[0] if (row["ceiling"] < minc or row["ceiling"] > maxc ) else( row["ceiling"]) ,axis=1) 
data_test['ceiling'] = data_test.apply(lambda row: data["ceiling"].mode()[0] if (row["ceiling"] < minc or row["ceiling"] > maxc ) else( row["ceiling"]) ,axis=1)  

data['ceiling'][data.ceiling.isna() ] = data["ceiling"].mode()[0]
data_test['ceiling'][data_test.ceiling.isna() ] = data["ceiling"].mode()[0]


#condition
var =  4.0
data['condition'] = data['condition'].fillna(var)
data_test['condition'] = data_test['condition'].fillna(var)


#stories
idss = data[["building_id"]][data.floor > data.stories].sort_values("building_id").drop_duplicates()
for i in range(idss.size):
    max_floor = data['floor'][data["building_id"] == idss["building_id"].iloc[i]].max()
    data['stories'][data["building_id"] == idss["building_id"].iloc[i]] =  max_floor

idss_test = data_test[["building_id"]][data_test.floor > data_test.stories].sort_values("building_id").drop_duplicates()
for i in range(idss_test.size):
    max_floor_test = data_test['floor'][data_test["building_id"] == idss_test["building_id"].iloc[i]].max()
    data_test['stories'][data_test["building_id"] == idss_test["building_id"].iloc[i]] =  max_floor_test


#material
data.material[data.material==5] = 2.0 #merging monlith brick with monolith
data.material[data.material==6] = 5.0 #stalin to 5

data_test.material[data_test.material==5] = 2.0
data_test.material[data_test.material==6] = 5.0

data['material'][data.material.isna() ] = data['material'].mode()[0]
data_test['material'][data_test.material.isna() ] = data['material'].mode()[0]

#constructed, new:
data['constructed'] = data.apply(
    lambda row: 2019 if (np.isnan(row['constructed']) and ~np.isnan(row['new']) and row['new'] == 0.0) else( 2021 if (np.isnan(row['constructed'])) else row['constructed']),
    axis=1
)  
data['new'] = data.apply(
    lambda row: 0.0 if (np.isnan(row['new']) and row['constructed'] < 2020) else( 1.0 if (np.isnan(row['new'])) else row['new']),
    axis=1
)      


data_test['constructed'] = data_test.apply(
    lambda row: 2019 if (np.isnan(row['constructed']) and ~np.isnan(row['new']) and row['new'] == 0.0) else( 2021 if (np.isnan(row['constructed'])) else row['constructed']),
    axis=1
)  
data_test['new'] = data_test.apply(
    lambda row: 0.0 if (np.isnan(row['new']) and row['constructed'] < 2020) else( 1.0 if (np.isnan(row['new'])) else row['new']),
    axis=1
)  




#FEATURE ENGINEERING:

lon1 =  37.621390
lat1 = 55.753098
geodesic = pyproj.Geod(ellps='WGS84')
distance_arr = []
back_azimuth_arr = []
fwd_azimuth_arr = []
for i in range(len(data["longitude"])):
    fwd_azimuth,back_azimuth,distance = geodesic.inv(lon1, lat1, data["longitude"][i], data["latitude"][i])
    distance_arr.append(distance)
    back_azimuth_arr.append(back_azimuth)
    fwd_azimuth_arr.append(fwd_azimuth)

data['fwd_azi'] = fwd_azimuth_arr
data['distance'] = distance_arr
data['back_azi'] = back_azimuth_arr


geodesic = pyproj.Geod(ellps='WGS84')
distance_arr = []
back_azimuth_arr = []
fwd_azimuth_arr = []
for i in range(len(data_test["longitude"])):
    fwd_azimuth,back_azimuth,distance = geodesic.inv(lon1, lat1, data_test["longitude"][i], data_test["latitude"][i])
    distance_arr.append(distance)
    back_azimuth_arr.append(back_azimuth)
    fwd_azimuth_arr.append(fwd_azimuth)

data_test['fwd_azi'] = fwd_azimuth_arr
data_test['distance'] = distance_arr
data_test['back_azi'] = back_azimuth_arr




data['area_per_room'] = data['area_total']/data['rooms']
data_test['area_per_room'] = data_test['area_total']/data_test['rooms']

data['area_per_room_log'] = np.log1p(data['area_per_room'])
data_test['area_per_room_log'] = np.log1p(data_test['area_per_room'])

data['area_total_log'] = np.log1p(data['area_total'])
data['area_kitchen_log'] = np.log1p(data['area_kitchen'])
data['area_living_log'] = np.log1p(data['area_living'])

data_test['area_total_log'] = np.log1p(data_test['area_total'])
data_test['area_kitchen_log'] = np.log1p(data_test['area_kitchen'])
data_test['area_living_log'] = np.log1p(data_test['area_living'])


data["bathrooms_total"] = data.bathrooms_shared + data.bathrooms_private
data_test["bathrooms_total"] = data_test.bathrooms_shared + data_test.bathrooms_private



#FEATURES INCLUDED:
features = ["ceiling", "area_per_room" ,  "area_per_room_log", "rooms", "area_total", "area_kitchen", "area_living", "area_total_log", "area_kitchen_log", "area_living_log", "floor", "new", "elevatern", "bathrooms_total", "bathrooms_shared", "bathrooms_private", 'parking', 'heating',
            "latitude", "longitude","district", "constructed", "condition", "seller", "total_balconies", "material", "stories",'distance','back_azi','fwd_azi']

    
#Model
train_x = data[features]
train_y = np.log1p(data['price']/data["area_total"])
test_x = data_test[features]



param = {
            'boosting_type': 'gbdt',
            'num_leaves': 35,
            'min_child_weight': 1,
            'subsample': 0.32415121173658534,
            'colsample_bytree':  0.4768205472451884,
            'reg_lambda': 0.14916991373512928,
            'reg_alpha': 0.006696476138868112,
            'learning_rate': 0.01747572661694792,
            'max_depth': 46,
            'n_estimators': 9775,
            'n_jobs' : 1,
            'objective' : 'regression',


        }


model_lgbm2 = LGBMRegressor(**param)




model_lgbm2.fit(train_x,train_y)

lgbm2_preds = model_lgbm2.predict(test_x)


In [ ]:
#Xgb2 Pipeline
train_x = data[features]
train_y = np.log1p(data['price']/data["area_total"])
test_x = data_test[features]

model_xgb2 = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
colsample_bytree=0.5728285533310635, gamma=0, learning_rate=0.016003653491882115, max_delta_step=0,
max_depth=8, min_child_weight=1, n_estimators=3515,
n_jobs=1, nthread=None, objective='reg:squarederror', random_state=5, # squarederror reg:squaredlogerror reg:squarederror
reg_alpha=0.005800171239325761, reg_lambda=0.48110648627756064, scale_pos_weight=1, seed=None, subsample=0.7155884414918227)


model_xgb2.fit(train_x,train_y)

xgb2_preds = model_xgb2.predict(test_x)



In [ ]:
#XGB3 (0.15178) on its own
apartments = pd.read_csv('data/apartments_train.csv')
buildings = pd.read_csv('data/buildings_train.csv')
data = pd.merge(apartments, buildings.set_index('id'), how='left', left_on='building_id', right_index=True)

apartments_test = pd.read_csv('data/apartments_test.csv')
buildings_test = pd.read_csv('data/buildings_test.csv')
data_test = pd.merge(apartments_test, buildings_test.set_index('id'), how='left', left_on='building_id', right_index=True)



import pyproj
from numpy.random import choice


#FEATURE CLEANING
data_test.latitude.iloc[90] = 55.568139
data_test.longitude.iloc[90]= 37.481831
data_test.latitude.iloc[23] = 55.568139
data_test.longitude.iloc[23]= 37.481831
data_test.latitude.iloc[2511] = 55.544066
data_test.longitude.iloc[2511]= 37.482317
data_test.latitude.iloc[6959] = 55.544066
data_test.longitude.iloc[6959]= 37.482317
data_test.latitude.iloc[5090] = 55.544066
data_test.longitude.iloc[5090]= 37.482317
data_test.latitude.iloc[8596] = 55.544066
data_test.longitude.iloc[8596]= 37.482317
data_test.latitude.iloc[2529] = 55.764335
data_test.longitude.iloc[2529]= 37.907556
data_test.latitude.iloc[4719] = 55.765430
data_test.longitude.iloc[4719]= 37.928284
data_test.latitude.iloc[9547] = 55.765430
data_test.longitude.iloc[9547]= 37.928284

data_test.district[data_test.building_id == 3803] = 11
data_test.district[data_test.building_id == 4636] = 11
data_test.district[data_test.building_id == 4412] = 11
data_test.district[data_test.building_id == 926] = 3
data_test.district[data_test.building_id == 4202] = 3
data_test.district[data_test.building_id == 8811] = 3
data_test.district[data_test.building_id == 6879] = 3
data_test.district[data_test.building_id == 5667] = 3
data_test.district[data_test.building_id == 2265] = 5
data_test.district[data_test.building_id == 6403] = 5
data_test.district[data_test.building_id == 7317] = 5
data_test.district[data_test.building_id == 1647] = 5
data_test.district[data_test.building_id == 183] = 5

data.district[data.building_id == 2029] = 0
data.district[data.building_id == 1255] = 0
data.district[data.building_id == 4162] = 5


#cleaning/engineering all elevators as one feature
data['elevatern'] = data.apply(lambda row: 0 if (row["elevator_without"] == 1 and row["elevator_passenger"] == 0.0 and row["elevator_service"] == 0.0 ) # 
                               else( 1 if(row["elevator_without"] == 1 and row["elevator_passenger"] == 0.0 and row["elevator_service"] == 1.0) # 
                                else(2 if(row["elevator_without"] == 1 and row["elevator_passenger"] == 1.0 and row["elevator_service"] == 0.0) # 
                                else(3 if(row["elevator_without"] == 0 and row["elevator_passenger"] == 0.0 and row["elevator_service"] == 1.0) # 
                                else(4 if(row["elevator_without"] == 0 and row["elevator_passenger"] == 1.0 and row["elevator_service"] == 0.0) # 
                                else(5 if(row["elevator_without"] == 0 and row["elevator_passenger"] == 1.0 and row["elevator_service"] == 1.0) # 
                                else(6 if(row["elevator_without"] == 1 and row["elevator_passenger"] == 1.0 and row["elevator_service"] == 1.0)
                                else(np.nan)
                                )    
                                )
                                )
                                )   
                                )
                                )
                                ,axis=1)
data_test['elevatern'] = data_test.apply(lambda row: 0 if (row["elevator_without"] == 1 and row["elevator_passenger"] == 0.0 and row["elevator_service"] == 0.0 ) # 
                               else( 1 if(row["elevator_without"] == 1 and row["elevator_passenger"] == 0.0 and row["elevator_service"] == 1.0) # 
                                else(2 if(row["elevator_without"] == 1 and row["elevator_passenger"] == 1.0 and row["elevator_service"] == 0.0) # 
                                else(3 if(row["elevator_without"] == 0 and row["elevator_passenger"] == 0.0 and row["elevator_service"] == 1.0) # 
                                else(4 if(row["elevator_without"] == 0 and row["elevator_passenger"] == 1.0 and row["elevator_service"] == 0.0) # 
                                else(5 if(row["elevator_without"] == 0 and row["elevator_passenger"] == 1.0 and row["elevator_service"] == 1.0) # 
                                else(6 if(row["elevator_without"] == 1 and row["elevator_passenger"] == 1.0 and row["elevator_service"] == 1.0)
                                else(np.nan)
                                ) 
                                )
                                )
                                )   
                                )
                                )
                                ,axis=1)  
mod = data['elevatern'].mode()
data['elevatern'] = data['elevatern'].fillna(mod[0])
data_test['elevatern'] = data_test['elevatern'].fillna(mod[0])


data["bathrooms_shared"] = data["bathrooms_shared"].fillna(1)
data["bathrooms_private"] = data["bathrooms_private"].fillna(1)
data_test["bathrooms_shared"] = data_test["bathrooms_shared"].fillna(1)
data_test["bathrooms_private"] = data_test["bathrooms_private"].fillna(1)



data["parking"] =  data["parking"].fillna(3.0)
data_test["parking"] =  data_test["parking"].fillna(3.0)
data["heating"] =  data["heating"].fillna(0.0)
data_test["heating"] =  data_test["heating"].fillna(0.0)

#engineering and cleaning a feature
data["total_balconies"] = data["balconies"] + data["loggias"]
data_test["total_balconies"] = data_test["balconies"] + data_test["loggias"]
data["total_balconies"] =  data["total_balconies"].fillna(1.0)
data_test["total_balconies"] =  data_test["total_balconies"].fillna(1.0)


#seller
list_of_candidates = [0,1,2,3]
probability_distribution  = [0.11, 0.33, 0.13, 0.43]
number_of_items_to_pick = data['seller'].isna().sum()
number_of_items_to_pick_test = data_test['seller'].isna().sum()

np.random.seed(0)

draw = choice(list_of_candidates, number_of_items_to_pick,
              p=probability_distribution)
draw_test = choice(list_of_candidates, number_of_items_to_pick_test,
              p=probability_distribution)

data['seller'][data.seller.isna()] = draw
data_test['seller'][data_test.seller.isna()] = draw_test


#area_kitchen and living
percentage_area_data = pd.DataFrame()
percentage_area_data["area_kitchen"] = data["area_kitchen"][data.area_living + data.area_kitchen < data.area_total]/data["area_total"][data.area_living + data.area_kitchen < data.area_total]
percentage_area_data["area_living"] = data["area_living"][data.area_living + data.area_kitchen < data.area_total]/data["area_total"][data.area_living + data.area_kitchen < data.area_total]

mean_kitchen = percentage_area_data["area_kitchen"].mean()
mean_living = percentage_area_data["area_living"].mean()

#to omit bugs
data["area_kitchen_edit"] = data["area_kitchen"].copy()
data["area_living_edit"] = data["area_living"].copy()

data["area_kitchen_edit"][(data.area_living + data.area_kitchen >= data.area_total) | (data.area_living.isna() | data.area_kitchen.isna())] = data.area_total*mean_kitchen
data["area_living_edit"][(data.area_living + data.area_kitchen >= data.area_total) | (data.area_living.isna() | data.area_kitchen.isna())] = data.area_total*mean_living

data["area_kitchen"] = data["area_kitchen_edit"].copy()
data["area_living"] = data["area_living_edit"].copy()

#test_set
data_test["area_kitchen_edit"] = data_test["area_kitchen"].copy()
data_test["area_living_edit"] = data_test["area_living"].copy()

data_test["area_kitchen_edit"][(data_test.area_living + data_test.area_kitchen >= data_test.area_total) | (data_test.area_living.isna() | data_test.area_kitchen.isna())] = data_test.area_total*mean_kitchen
data_test["area_living_edit"][(data_test.area_living + data_test.area_kitchen >= data_test.area_total) | (data_test.area_living.isna() | data_test.area_kitchen.isna())] = data_test.area_total*mean_living


data_test["area_kitchen"] = data_test["area_kitchen_edit"].copy()
data_test["area_living"] = data_test["area_living_edit"].copy()
        

#ceiling    
maxc = 9
minc = 1
data['ceiling'] = data.apply(lambda row: data["ceiling"].mode()[0] if (row["ceiling"] < minc or row["ceiling"] > maxc ) else( row["ceiling"]) ,axis=1) 
data_test['ceiling'] = data_test.apply(lambda row: data["ceiling"].mode()[0] if (row["ceiling"] < minc or row["ceiling"] > maxc ) else( row["ceiling"]) ,axis=1)  

data['ceiling'][data.ceiling.isna() ] = data["ceiling"].mode()[0]
data_test['ceiling'][data_test.ceiling.isna() ] = data["ceiling"].mode()[0]


#condition
var =  4.0
data['condition'] = data['condition'].fillna(var)
data_test['condition'] = data_test['condition'].fillna(var)


#stories
idss = data[["building_id"]][data.floor > data.stories].sort_values("building_id").drop_duplicates()
for i in range(idss.size):
    max_floor = data['floor'][data["building_id"] == idss["building_id"].iloc[i]].max()
    data['stories'][data["building_id"] == idss["building_id"].iloc[i]] =  max_floor

idss_test = data_test[["building_id"]][data_test.floor > data_test.stories].sort_values("building_id").drop_duplicates()
for i in range(idss_test.size):
    max_floor_test = data_test['floor'][data_test["building_id"] == idss_test["building_id"].iloc[i]].max()
    data_test['stories'][data_test["building_id"] == idss_test["building_id"].iloc[i]] =  max_floor_test


#material
data.material[data.material==5] = 2.0 #merging monlith brick with monolith
data.material[data.material==6] = 5.0 #stalin to 5

data_test.material[data_test.material==5] = 2.0
data_test.material[data_test.material==6] = 5.0

data['material'][data.material.isna() ] = data['material'].mode()[0]
data_test['material'][data_test.material.isna() ] = data['material'].mode()[0]

#constructed, new:
data['constructed'] = data.apply(
    lambda row: 2019 if (np.isnan(row['constructed']) and ~np.isnan(row['new']) and row['new'] == 0.0) else( 2021 if (np.isnan(row['constructed'])) else row['constructed']),
    axis=1
)  
data['new'] = data.apply(
    lambda row: 0.0 if (np.isnan(row['new']) and row['constructed'] < 2020) else( 1.0 if (np.isnan(row['new'])) else row['new']),
    axis=1
)      


data_test['constructed'] = data_test.apply(
    lambda row: 2019 if (np.isnan(row['constructed']) and ~np.isnan(row['new']) and row['new'] == 0.0) else( 2021 if (np.isnan(row['constructed'])) else row['constructed']),
    axis=1
)  
data_test['new'] = data_test.apply(
    lambda row: 0.0 if (np.isnan(row['new']) and row['constructed'] < 2020) else( 1.0 if (np.isnan(row['new'])) else row['new']),
    axis=1
)  




#FEATURE ENGINEERING:

lon1 =  37.621390
lat1 = 55.753098
geodesic = pyproj.Geod(ellps='WGS84')
distance_arr = []
back_azimuth_arr = []
fwd_azimuth_arr = []
for i in range(len(data["longitude"])):
    fwd_azimuth,back_azimuth,distance = geodesic.inv(lon1, lat1, data["longitude"][i], data["latitude"][i])
    distance_arr.append(distance)
    back_azimuth_arr.append(back_azimuth)
    fwd_azimuth_arr.append(fwd_azimuth)

data['fwd_azi'] = fwd_azimuth_arr
data['distance'] = distance_arr
data['back_azi'] = back_azimuth_arr


geodesic = pyproj.Geod(ellps='WGS84')
distance_arr = []
back_azimuth_arr = []
fwd_azimuth_arr = []
for i in range(len(data_test["longitude"])):
    fwd_azimuth,back_azimuth,distance = geodesic.inv(lon1, lat1, data_test["longitude"][i], data_test["latitude"][i])
    distance_arr.append(distance)
    back_azimuth_arr.append(back_azimuth)
    fwd_azimuth_arr.append(fwd_azimuth)

data_test['fwd_azi'] = fwd_azimuth_arr
data_test['distance'] = distance_arr
data_test['back_azi'] = back_azimuth_arr




data['area_per_room'] = data['area_total']/data['rooms']
data_test['area_per_room'] = data_test['area_total']/data_test['rooms']

data['area_per_room_log'] = np.log1p(data['area_per_room'])
data_test['area_per_room_log'] = np.log1p(data_test['area_per_room'])

data['area_total_log'] = np.log1p(data['area_total'])
data['area_kitchen_log'] = np.log1p(data['area_kitchen'])
data['area_living_log'] = np.log1p(data['area_living'])

data_test['area_total_log'] = np.log1p(data_test['area_total'])
data_test['area_kitchen_log'] = np.log1p(data_test['area_kitchen'])
data_test['area_living_log'] = np.log1p(data_test['area_living'])


data["bathrooms_total"] = data.bathrooms_shared + data.bathrooms_private
data_test["bathrooms_total"] = data_test.bathrooms_shared + data_test.bathrooms_private




#floor/stories
data["floor/stories"] = data["floor"]/data["stories"]
data_test["floor/stories"] = data_test["floor"]/data_test["stories"]


#euclidean financial distance from city center
financial_coords = (37.535497858, 55.741330368)
distance_from_city_center = np.sqrt((financial_coords[0] - data["longitude"])**2+(financial_coords[1] - data["latitude"])**2)
data["distance_from_financial_center"] = distance_from_city_center

distance_from_city_center_t = np.sqrt((financial_coords[0] - data_test["longitude"])**2+(financial_coords[1] - data_test["latitude"])**2)
data_test["distance_from_financial_center"] = distance_from_city_center_t


#euclidean distance from city center
origin_coordinates = (37.621390,55.753098)
distance_from_city_center = np.sqrt((origin_coordinates[0] - data["longitude"])**2+(origin_coordinates[1] - data["latitude"])**2)
data["distance_from_city_center"] = distance_from_city_center

distance_from_city_center_t = np.sqrt((origin_coordinates[0] - data_test["longitude"])**2+(origin_coordinates[1] - data_test["latitude"])**2)
data_test["distance_from_city_center"] = distance_from_city_center_t

#FEATURES INCLUDED:
features = ["ceiling", "area_per_room" ,  "area_per_room_log", "rooms", "area_total", "area_kitchen", "area_living", "area_total_log", "area_kitchen_log", "area_living_log",
            "floor", "new", "elevatern", "bathrooms_total", "bathrooms_shared", "bathrooms_private", 'parking', 'heating',
            "latitude", "longitude","district", "constructed", "condition", "seller", "total_balconies", "material", "stories",'distance','back_azi','fwd_azi',"floor/stories",
           "distance_from_financial_center", "distance_from_city_center"]





#MODEL:
train_x = data[features]
train_y = np.log1p(data['price']/data["area_total"])
test_x = data_test[features]

param = {
        'base_score' : 0.5,
        'booster' : 'gbtree',
        'colsample_bylevel' : 1,
        'gamma' : 0,
        'max_delta_step' : 0,
        'n_jobs' : -1,
        'nthread' : None,
        'objective' : 'reg:squarederror',
        'scale_pos_weight' : 1,
        'seed' : None,
        'lambda': 0.0024064014952485785, 
         'alpha': 0.001541503784279617, 
        'colsample_bytree': 0.43152225018148443, 
       'subsample': 0.8078473020517652, 
       'learning_rate': 0.013367834721822036, 
       'n_estimators': 5235, 
     'random_state': 291, 
      'max_depth': 9, 
    'min_child_weight': 13
}



model_xgb3 = XGBRegressor(**param)

model_xgb3.fit(train_x,train_y)

xgb3_preds = model_xgb3.predict(test_x)


In [ ]:
#Lgbm3 pipeline 0.15912 on test
#MODEL
train_x = data[features]
train_y = np.log1p(data['price']/data["area_total"])
test_x = data_test[features]




best_params = {'objective' : 'regression',
    "metric": "root_mean_squared_error",
    'random_state': 2020,
    "n_estimators": 3000,
    'boosting_type': 'gbdt', #better than dart
    "n_jobs": -1,
 'learning_rate': 0.009902216010560466, 
 'num_iterations': 9853, 
 'n_estimators': 2200, 
 'max_bin': 1145, 
 'num_leaves': 992, 
 'min_data_in_leaf': 21, 
 'min_sum_hessian_in_leaf': 6, 
 'bagging_fraction': 0.7553160099162841, 
 'bagging_freq': 1, 
 'max_depth': 5, 
 'lambda_l1': 0.001047756084491848, 
 'lambda_l2': 0.5231817241800534, 
 'min_gain_to_split': 0.01715842845568677
    }




model_lgbm3 = LGBMRegressor(**best_params)  

model_lgbm3.fit(train_x,train_y,verbose=False)

lgbm3_preds = model_lgbm3.predict(test_x)

In [ ]:
#catb3 pipeline 0.15484
from catboost import CatBoostRegressor

#MODEL:
train_x = data[features]
train_y = np.log1p(data['price']/data["area_total"])
test_x = data_test[features]

 


param = {
"objective": "RMSE",
'depth': 8, 
 'reg_lambda': 0.6424630162452156, 
 'learning_rate': 0.008856338969505724, 
 'n_estimators': 5356, 
 'max_bin': 1042, 
 'random_state': 1695, 
 'subsample': 0.4474582804576312}




model_catb3 = CatBoostRegressor(**param)  

model_catb3.fit(train_x,train_y,early_stopping_rounds=100,verbose=False)

catb3_preds = model_catb3.predict(test_x)

In [ ]:
#Stacked Model Pipeline
train_x = data[features]
train_y = np.log1p(data['price']/data["area_total"])
test_x = data_test[features]



from mlxtend.regressor import StackingCVRegressor

stacked_model = StackingCVRegressor(regressors=(model_xgb1, model_xgb2, model_cat2, model_lgbm3,model_xgb3,model_catb3),
                                meta_regressor=model_xgb3, #our best individual model becomes the META
                                use_features_in_secondary=True,
                                   verbose=0)



stacked_model.fit(train_x,train_y)

stacked_preds = stacked_model.predict(test_x)

In [ ]:
#Weighted Averaging/Blending Model Pipeline
final_preds = np.average(
    [np.expm1(xgb_preds),
     np.expm1(xgb2_preds)*data_test["area_total"],
     np.expm1(stacked_preds)*data_test["area_total"],
     np.expm1(catboost2_preds),
     np.expm1(lgbm3_preds)*data_test["area_total"],
     np.expm1(xgb3_preds)*data_test["area_total"],
     np.expm1(catb3_preds)*data_test["area_total"]
    ],
    weights = 1 / np.array([0.19558,  0.18874, 0.19250,0.19819,0.18979,0.19083,0.19596]) ** 6,  #Should be 4 by standard and then increase to 6 to squeeze more juice
    axis=0
)




In [ ]:
#Submission

# Construct submission dataframe
submission = pd.DataFrame()
submission['id'] = data_test.id
submission['price_prediction'] = final_preds # *0.99839    
print(f'Generated {len(submission)} predictions')

# Export submission to csv with headers
submission.to_csv('submission.csv', index=False)

# Look at submitted csv
print('\nLine count of submission')
!wc -l submission.csv

print('\nFirst 5 rows of submission')
!head -n 10 submission.csv